In [1]:
import os
import pandas as pd
import numpy as np
import nipy
from mvpa2.suite import * # Production code should import functions individually.
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn import metrics # For calculating AUROC
from sklearn.linear_model.logistic import LogisticRegression 
from sklearn import neighbors
from sklearn.svm import SVC

#### Load Data

In [3]:
def load_data(subject_number):
    """Returns an fmri dataset object containing data of all 10 runs for a particular subject."""
    
    cwd = '/home/fmri-data'
    data_path = os.path.join(cwd,"mvpa_"+str(subject_number))

    # Load the sample attributes
    attr_fname = "targets_attr_mvpa_"+str(subject_number)+".txt"
    attr_path = os.path.join(data_path, attr_fname)
    attr = SampleAttributes(attr_path) # PyMVPA convenience function to read attribute files.

    # Load all 10 runs from one subject into one fMRI dataset
    file_path_list = list() # file path list to each run
    for i in range(1,11):
        run_name = "run"+str(i)
        fname = run_name +"_native.nii.gz"
        filepath = os.path.join(data_path, fname)
        file_path_list.append(filepath)

    # fMRI dataset format is convenient and vital for using PyMVPA.
    ds = fmri_dataset(file_path_list,targets=attr.targets,chunks=attr.chunks)
    return ds

In [4]:
# for subject_number in range(1,43):
        # Load the BOLD data for all 10 runs of a given subject
ds = load_data(1)

 * Please note: warnings are printed only once, but underlying problem might occur many times *


In [5]:
ds_cp = ds.copy(deep=False,
                       sa=['targets', 'chunks'],
                       fa=['voxel_indices'],
                       a=['mapper'])

In [ ]:
clf = kNN(k=1, dfx=one_minus_correlation, voting='majority')
cv = CrossValidation(clf, HalfPartitioner())
sl = sphere_searchlight(cv, radius=3, postproc=mean_sample())
sl_map = sl(ds_cp)

/root/anaconda2/lib/python2.7/site-packages/mvpa2/clfs/distance.py:257: RuntimeWarning: divide by zero encountered in divide
  Zx /= np.c_[X.std(axis=1)]
/root/anaconda2/lib/python2.7/site-packages/mvpa2/clfs/distance.py:257: RuntimeWarning: invalid value encountered in divide
  Zx /= np.c_[X.std(axis=1)]
/root/anaconda2/lib/python2.7/site-packages/mvpa2/clfs/distance.py:259: RuntimeWarning: divide by zero encountered in divide
  Zy /= np.c_[Y.std(axis=1)]
/root/anaconda2/lib/python2.7/site-packages/mvpa2/clfs/distance.py:259: RuntimeWarning: invalid value encountered in divide
  Zy /= np.c_[Y.std(axis=1)]


In [ ]:
# write out to nifti image
niftiresults = map2nifti(sl_map, imghdr=ds.a.imghdr)
niftiresults.to_filename('mytest_sl_imghdr.nii.gz')